In [124]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, ShuffleSplit, GridSearchCV
from ngboost import NGBRegressor
from ngboost.distns import Poisson, LogNormal, Normal, Laplace
import shap
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = [20, 10]

In [125]:
df = pd.read_parquet("data/processed.parquet")

In [126]:
X = df.drop(columns="Avg_Salary").values
y = df["Avg_Salary"].values

In [ ]:
Dist_list = [Poisson, LogNormal, Normal, Laplace]
param_grid_list = []

param_grid = {
    'n_estimators': [100, 500, 1000],
    'minibatch_frac': [1.0, 0.5],
    'learning_rate': [0.01, 0.001]
}

for Dist in Dist_list
    model = NGBRegressor(Dist=LogNormal)
    grid_search = GridSearchCV(model, param_grid=param_grid, cv=5)
    grid_search.fit(X, y)
    grid_search_list.append(grid_search)

In [ ]:
model.fit(X, y)

In [ ]:
model.predict(X)

In [ ]:
y_pred = model.pred_dist(X)

In [ ]:
from scipy.stats import lognorm

In [ ]:
np.exp(np.log(mu) + s**2/2)

In [ ]:
y_pred.params

In [ ]:
model.pred_dist(X.iloc[0:1])

In [ ]:
y_dists.params["s"]

In [ ]:
# Predict
y_dists = model.pred_dist(X.iloc[4:5])
#mu  = np.round(y_dists.params["loc"])/1000
#sigma = np.round(y_dists.params["scale"])/1000

In [ ]:
mu  = np.log(y_dists.params["scale"])
s  = y_dists.params["s"]

In [ ]:
x = np.linspace(1, 80000, 1000)
pdf = 1/(x * s * np.sqrt(2*np.pi)) * np.exp(-((np.log(x)-mu)**2)/2*s**2)

In [ ]:
lognorm.cdf(x[0], s,scale=np.exp(mu))

In [ ]:
lognorm.pdf(x, s,scale=np.exp(mu))

In [ ]:
pdf

In [ ]:
file_path = Path("data/model.p")

with file_path.open("wb") as f:
    pickle.dump(model, f)

In [ ]:
with file_path.open("rb") as f:
    model = pickle.load(f)

# Model Analysis

In [ ]:
explainer = shap.TreeExplainer(model, model_output=0)
shap_values = explainer.shap_values(X)

In [ ]:
shap.summary_plot(shap_values, X)

In [ ]:
cols = df.columns[df.columns.str.contains("Language")]

In [ ]:
df.groupby("Job_Role_Original")[cols].sum().sum(axis=1).values

In [ ]:
100*df.groupby("Job_Role_Original")[cols].count().divide(df.groupby("Job_Role_Original")[cols].sum().sum(axis=1).values, axis=0).round(3)["Language_Lua"]

In [ ]:
df.groupby("Job_Role_Original").Language_Perl.sum()

In [ ]:
df["Language_SQL"]

In [ ]:
shap.dependence_plot("Job_Role_Original", shap_values, X)

In [ ]:
df.Language_SQL.value_counts()

In [ ]:
df[df.Language_R==1].Working_Experience.mean()

In [ ]:
df[df.Language_R==0].Working_Experience.mean()

In [ ]:
df[(df.Language_R==1) & (df.Work_Company_Country=="Portugal")].Avg_Salary.mean()

In [ ]:
df[(df.Language_R==0) & (df.Work_Company_Country=="Portugal")].Avg_Salary.mean()

In [ ]:
import shap

# train an XGBoost model
X, y = shap.datasets.boston()
model = LGBMRegressor().fit(X, y)

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn, transformers, Spark, etc.)
explainer = shap.Explainer(model)
shap_values = explainer(X)

# visualize the first prediction's explanation
shap.plots.waterfall(shap_values[0])

In [ ]:
shap.dependence_plot("Gold earned per min.", shap_values, X, interaction_index="Deaths per min.")